In [1]:
from pyprojroot.here import here
import sys
import pandas as pd
import pathlib
from collections import defaultdict, Counter
import copy

sys.path.append(str(here()))

from resources.utils import *
from test_utils import *

In [2]:
# def check_setup_usernames(usernames):
#     """
#     Checks that all usernames are unique. 
#     Raises a value error if not.

#     Parameters
#     ----------
#     usernames : list of usernames
#     """
#     dupes = [item for item, count in Counter(usernames).items() if count > 1]
#     if len(dupes)>0:
#         raise ValueError(f'Found duplicated username {dupes}.')
        
# def verify_proj_name(name):
#     """
#     Checks if name is still the template name
#     Raises a value error if it is

#     Parameters
#     ----------
#     name : str
#     """
#     if name == 'project_template':
#         raise ValueError(f'Must provide a new project name in template_user/resources.yml!')

# def construct_templated_paths(path_map, path, user, username, proj_name):
#     """
#     Generate the paths that we can systematically infer
#     to make the user input minimal.
#     1. all paths relative to the project (ref/, figures/, data/, user/metadata)
#     2. for mn5, links to scratch and projects
    
#     This implementation, in theory makes this easy for mn5 users,
#     while also allowing for use on other systems as well, if the user
#     puts all the *_dir paths in their setup
#     """
    
#     pref = f"{path}/{proj_name}/"

#     data_dir = f'{pref}/data/'
#     ref_dir = f'{pref}/ref/'
#     figures_dir = f'{pref}/figures/'

#     # metadata dir is part of the github-stored stuff, so it's separate
#     metadata_dir = str(pathlib.Path(f'{pref}/{user}/metadata/'))

#     # add all paths to dict
#     path_map[username]['data_dir'] = str(pathlib.Path(data_dir))
#     path_map[username]['ref_dir'] = str(pathlib.Path(ref_dir))
#     path_map[username]['figures_dir'] = str(pathlib.Path(figures_dir))
#     path_map[username]['metadata_dir'] = str(pathlib.Path(metadata_dir))
    
#     return path_map


# def generate_path_map(m, proj_name):
#     """
#     Parse the setup_settings dictionary.
    
#     Returns
#     --------
#     path_map : dict, {username: [list of paths the user cares about],
#                       users: [list of user aliases]}
#     """
    
#     # create the path map entries automatically
#     path_map = defaultdict(dict)
#     for user, entry in m['setup_settings']['users'].items():
#         for system, entry2 in entry.items():
#             # import pdb; pdb.set_trace()

#             for k, i in entry2.items():
#                 if k.endswith('_dir'):
#                     path_map[entry2['username']][k] = i
            
#             # for both of these, append /Projects/ so we don't have to keep track of both
#             # /gpfs/projects/bsc83/ and /gpfs/projects/bsc83/Projects
#             if system == 'mn5':
#                 # for mn5, we record the projects path already in the config
#                 path = f"{m['setup_settings']['mn5_locs']['projects_dir']}/Projects/"
#                 # as well as the default projects and scratch dirs
#                 for mn5_dir in ['projects_dir', 'scratch_dir']:
#                     path_map[entry2['username']][mn5_dir] = m['setup_settings']['mn5_locs'][mn5_dir]
#             else:
#                 path = f"{m['setup_settings']['users'][user][system]['projects_dir']}/Projects/"

#             # add the templated directories -- ones we know where to find either
#             # 1. relative to the project or 
#             # 2. based on abs. paths on mn5
#             path_map = construct_templated_paths(path_map,
#                                       path,
#                                       user, 
#                                       entry2['username'],
#                                       proj_name)
            
#             # if mn5, also add all these paths as mn5 user
#             if system == 'mn5':
#                 # import pdb; pdb.set_trace()
#                 path_map['mn5_user'] = copy.deepcopy(path_map[entry2['username']])
            
#     path_map = dict(path_map)

#     return path_map

In [3]:
def main(test=True, config_file=None,
         output_config='template_user/resources/resources.yml'):
    m = load_resources(config_file)
    
    proj_name = m['setup_settings']['project_name']
    
    verify_proj_name(proj_name)
    
    usernames = [i2['username'] for k, i in m['setup_settings']['users'].items() for k2, i2 in i.items()]
    check_setup_usernames(usernames)
    
    # finally a way to test this
    if not test:
        # rename project; immediately remove all git things;
        cmd = 'rm -rf .git'
        run_cmd(cmd)
        cmd = f"mv ../project_template ../{proj_name}"
        run_cmd(cmd)

    path_map = generate_path_map(m, proj_name)
    path_map = {'path_map': path_map}
    
    # also add a users list
    users_list = {}
    users_list['users'] = list(m['setup_settings']['users'].keys())

    # write to resources.yml, just append the path_map and quick-access path maps
    if test: open_mode = 'w'
    else: open_mode = 'a'
    with open(output_config, open_mode) as f:
        yaml.dump(path_map, f, default_flow_style=False)
        yaml.dump(users_list, f, default_flow_style=False)
    
    # make a copy of template user for each user
    if not test:
        for user, systems in m['setup_settings']['users'].items():
            cmd = f'cp -r template_user/ {user}'
            run_cmd(cmd)

In [4]:
m = main(test=True, 
         config_file='test_yml.yml',
        output_config='test_yml_out.yml')